API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangSmith 추적을 설정합니다.(선택)

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-EMBEDDING")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-EMBEDDING


In [2]:
# 라이브러리 선언
import sqlite3
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import re

In [9]:
# 1. 데이터베이스 연결 및 조회 (SQLite 일 경우)
sql = "SELECT * FROM CATEGORY"
# SQLite 데이터베이스 연결
conn = sqlite3.connect('../.db/sqlite/my_database.db')
cursor = conn.cursor()

# SQL 쿼리 실행
cursor.execute(sql)
rows = cursor.fetchall()

# 샘플 출력
for row in rows[:10]:
    print(row)

# 연결 종료
cursor.close()
conn.close()

(1, '음향가전', '스피커', '블루투스스피커')
(2, '음향가전', '이어폰', '무선이어폰')
(3, '음향가전', '기타', '')
(4, '음향가전', '헤드폰', '무선헤드폰')
(5, '음향가전', '헤드폰', '유선헤드폰')
(6, '생활·주방용품', '주방용품', '그릇·식기')
(7, '냉장고·주방가전', '에어프라이어 · 튀김기', '')
(8, '음향가전', '스피커', '홈시어터·사운드바')
(9, '음향가전', '', '')
(10, '음향가전', '이어폰', '유선이어폰')


In [13]:
# 2. 주어진 컬럼들을 바탕으로 ML 검색에 용이하도록 문맥에 맞게 병합하는 함수 정의

# 이 부분은 실제 데이터의 컬럼 순서와 일치해야 합니다.
COLUMN_INDICES = {
    "ID": 0,
    "LGRP_NM": 1,
    "MGRP_NM": 2,
    "SGRP_NM": 3
}

def generate_search_text_from_tuple(category_data_tuple):

    # 튜플에서 데이터 추출 (COLUMN_INDICES를 활용하여 가독성을 높입니다)
    lgrp_nm = category_data_tuple[COLUMN_INDICES["LGRP_NM"]].strip()
    mgrp_nm = category_data_tuple[COLUMN_INDICES["MGRP_NM"]].strip()
    sgrp_nm = category_data_tuple[COLUMN_INDICES["SGRP_NM"]].strip()
    
    # 카테고리 이름 중 비어있지 않은 부분만 '>' 구분자로 연결합니다.
    parts = [part for part in [lgrp_nm, mgrp_nm, sgrp_nm] if part]
    return ">".join(parts)

In [14]:
# 3. 상품정보를 문맥에 맞게 병합한 `search_text` 를 포함하여 documents 리스트 생성

documents = []
metadatas = []
ids = []

for row in rows:
    ID, LGRP_NM, MGRP_NM, SGRP_NM = row
    metadatas.append(
        {"LGRP_NM": LGRP_NM, 
         "MGRP_NM": MGRP_NM, 
         "SGRP_NM": SGRP_NM
         })
    search_text = generate_search_text_from_tuple(row)
    documents.append(search_text)

# 샘플 출력
for i, doc in enumerate(documents[:10]):
    print(f"{i+1}. {doc}")


1. 음향가전>스피커>블루투스스피커
2. 음향가전>이어폰>무선이어폰
3. 음향가전>기타
4. 음향가전>헤드폰>무선헤드폰
5. 음향가전>헤드폰>유선헤드폰
6. 생활·주방용품>주방용품>그릇·식기
7. 냉장고·주방가전>에어프라이어 · 튀김기
8. 음향가전>스피커>홈시어터·사운드바
9. 음향가전
10. 음향가전>이어폰>유선이어폰


In [15]:
# 4. 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

한 번 요청에 임베딩 할 수 있는 토큰 수가 한정되어 있기 때문에 상품 개수를 잘라서 처리합니다.

In [16]:
# 5. DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.

# DB 경로 설정
persist_directory = "../.db/category"

# 배치 크기 설정
BATCH_SIZE = 500
total_products = len(documents)

# 첫 번째 배치로 FAISS 초기화
print(f"처리 중: 1~{min(BATCH_SIZE, total_products)}번째 상품 / 총 {total_products}개")
first_batch_docs = documents[:BATCH_SIZE]
first_batch_metadatas = metadatas[:BATCH_SIZE]
first_batch_ids = ids[:BATCH_SIZE]

vectorstore = FAISS.from_texts(
    texts=first_batch_docs,
    embedding=embeddings,
    metadatas=first_batch_metadatas,
    ids=first_batch_ids
)

# 나머지 배치 처리
for i in range(BATCH_SIZE, total_products, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, total_products)
    print(f"처리 중: {i+1}~{end_idx}번째 상품 / 총 {total_products}개")
    
    batch_docs = documents[i:end_idx]
    batch_metadatas = metadatas[i:end_idx]
    batch_ids = ids[i:end_idx]
    
    try:
        vectorstore.add_texts(
            texts=batch_docs,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
    except Exception as e:
        print(f"배치 처리 중 오류 발생: {str(e)}")
        continue

# 로컬에 저장
vectorstore.save_local(persist_directory)

print(f"총 {total_products}개의 상품 정보가 FAISS에 저장되었습니다.")

처리 중: 1~500번째 상품 / 총 542개
처리 중: 501~542번째 상품 / 총 542개
총 542개의 상품 정보가 FAISS에 저장되었습니다.


임베딩 결과를 확인합니다.

In [17]:

# 6. 샘플 출력

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    k=1  # 반환할 결과 수
)

# 벡터스토어에서 임베딩된 데이터 확인
print("\n[벡터스토어 임베딩 데이터]")
print("-" * 60)
print(f"저장된 총 문서 수: {vectorstore.index.ntotal}")

# 문맥 샘플 확인
print(f"\n첫 번째 문서의 문맥 값:")
if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"{doc.page_content}")

# 임베딩 벡터 샘플 확인
sample_vector = vectorstore.index.reconstruct(0)  # 첫 번째 문서의 임베딩 벡터
print(f"\n첫 번째 문서의 임베딩 벡터 (처음 10개 값):")
print(sample_vector[:10])
print("-" * 60)


[벡터스토어 임베딩 데이터]
------------------------------------------------------------
저장된 총 문서 수: 542

첫 번째 문서의 문맥 값:
TV·영상가전

첫 번째 문서의 임베딩 벡터 (처음 10개 값):
[-0.00417116 -0.00360547 -0.02399505 -0.08265237  0.00327601 -0.01573976
  0.01166184  0.08429348  0.00653337 -0.03110653]
------------------------------------------------------------


In [39]:
query = "홈카페에 어울리는 세련된 디자인의 커피머신"  # 예시 쿼리: 원하는 검색어로 변경하세요

results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    k=10  # 반환할 결과 수
)

for i, (doc, score) in enumerate(results_with_score):
    # print(f"{i}: {doc.metadata.get('LGRP_NM', '정보 없음')} > {doc.metadata.get('MGRP_NM', '정보 없음')} > {doc.metadata.get('SGRP_NM', '정보 없음')}")
    print(f"{i}: {doc.page_content}")   

0: 가구·인테리어>패브릭>카페트·매트
1: 냉장고·주방가전>커피머신>반자동커피머신
2: 냉장고·주방가전>커피머신>커피용품
3: 냉장고·주방가전>커피머신>커피메이커
4: 컴퓨터·노트북>전시상품>데스크탑
5: 홈앤라이프샵
6: 생활·주방용품>주방용품>커피·티·와인용품
7: 가구·인테리어>홈데코>기타홈데코
8: 냉장고·주방가전>커피머신>캡슐커피머신
9: 스마트홈>홈카메라(CCTV)


**매개변수(parameters)**

- `**kwargs`: 검색 함수에 전달할 키워드 인자
  - `search_type`: 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
  - `search_kwargs`: 추가 검색 옵션
    - `k`: 반환할 문서 수 (기본값: 4)
    - `score_threshold`: similarity_score_threshold 검색의 최소 유사도 임계값
    - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
    - `lambda_mult`: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
    - `filter`: 문서 메타데이터 기반 필터링

In [91]:
query = "홈카페 디자인 커피머신"  # 예시 쿼리: 원하는 검색어로 변경하세요

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", #similarity, similarity_score_threshold, mmr
    search_kwargs={
        "k": 11,
        "score_threshold": 0.09
        # lambda_mult:0.1
    }
)

results = retriever.invoke(query)

for i, doc in enumerate(results):
    # print(f"{i}: {doc.metadata.get('LGRP_NM', '정보 없음')} > {doc.metadata.get('MGRP_NM', '정보 없음')} > {doc.metadata.get('SGRP_NM', '정보 없음')}")
    print(f"{i}: {doc.page_content}")

0: 냉장고·주방가전>커피머신>반자동커피머신
1: 가구·인테리어>패브릭>카페트·매트
2: 냉장고·주방가전>커피머신>커피메이커
3: 스마트홈>홈카메라(CCTV)
4: 냉장고·주방가전>커피머신>커피용품
5: 냉장고·주방가전>커피머신>캡슐커피머신
6: 냉장고·주방가전>커피머신>전자동커피머신
7: 홈앤라이프샵
8: Premium Kitchen
